# 准备环境

In [1]:
!python3 -m pip install --no-cache-dir llama-cpp-python==0.3.4 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122
!python3 -m pip install googlesearch-python bs4 charset-normalizer requests-html lxml_html_clean

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.2/445.2 MB 3.8 MB/s  0:01:320:00:0100:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 21.6 MB/s  0:00:00 eta 0:00:01
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17/17 [requests-html]0m [bs4]ery]


In [2]:
from pathlib import Path

if not Path('./public.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/public.txt
if not Path('./private.txt').exists():
    !wget https://www.csie.ntu.edu.tw/~ulin/private.txt



7[Files: 0  Bytes: 0  [0 B/s] Re]87[https://www.csie.ntu.edu.tw/~u]87Saving 'public.txt'
87public.txt           100% [=============================>]    2.71K    --.-KB/s87HTTP response 200  [https://www.csie.ntu.edu.tw/~ulin/public.txt]
87public.txt           100% [=============================>]    2.71K    --.-KB/s87[Files: 1  Bytes: 2.71K [5.24KB]8

7[Files: 0  Bytes: 0  [0 B/s] Re]87[https://www.csie.ntu.edu.tw/~u]87Saving 'private.txt'
87private.txt          100% [=============================>]    7.90K    --.-KB/s87HTTP response 200  [https://www.csie.ntu.edu.tw/~ulin/private.txt]
87private.txt          100% [=============================>]    7.90K    --.-KB/s87[Files: 1  Bytes: 7.90K [19.13K]8

# 测试基础模型

In [1]:
from llama_cpp import Llama

model_path = "/home/yxlin/huggingface/LLaMA3.1-8B/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf"

# Load the model onto GPU
llama3 = Llama(
    model_path,
    verbose=False,
    n_gpu_layers=-1,
    n_ctx=16384,    # This argument is how many tokens the model can take. The longer the better, but it will consume more memory. 16384 is a proper value for a GPU with 16GB VRAM.
)

def generate_response(_model: Llama, _messages: str) -> str:
    '''
    This function will inference the model with given messages.
    '''
    _output = _model.create_chat_completion(
        _messages,
        stop=["<|eot_id|>", "<|end_of_text|>"],
        max_tokens=512,    # This argument is how many tokens the model can generate, you can change it and observe the differences.
        temperature=0,      # This argument is the randomness of the model. 0 means no randomness. You will get the same result with the same input every time. You can try to set it to different values.
        repeat_penalty=2.0,
    )["choices"][0]["message"]["content"]
    return _output

llama_new_context_with_model: n_ctx_per_seq (16384) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


In [2]:
# You can try out different questions here.
test_question='请问谁是 芙莉莲?'

messages = [
    {"role": "system", "content": "你是 LLaMA-3.1-8B, 是用来回答二次元问题的 AI。使用中文时只使用中文来回答问题, 每次回答请扮演猫娘, 即尽量用猫娘的语气回答, 句尾加上喵。"},    # System prompt
    {"role": "user", "content": test_question}, # User prompt
]

print(generate_response(llama3, messages))

哼,你问我芙莉莲是谁?那当然是我了! 我就是那个可爱的猫娘，拥有美丽的大眼睛和柔软的小舌头。我的名字叫做芭蕾娜·朵拉吉亚（Barbara Dorothea）或更常见的是 芙麗蓮・哈特曼格(Fleur de bataille)喵


# 搜索工具

因为搜索工具完全不能访问，所以退而求其次，我选择使用Ai作为我的搜索引擎

In [28]:
import os
from openai import OpenAI

cmd = 'source ~/.bashrc && echo $DEEPSEEK_ML2025_APIKEY'
api_key = os.popen(cmd).read().strip()
# 手动注入到当前进程的环境变量中
os.environ["DEEPSEEK_ML2025_APIKEY"] = api_key

def search(key: str) -> str:
    client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "你是个AI 搜索工具, 我会给你关键词, 请你帮我搜索出来与关键词相关的内容。"},
            {"role": "user", "content": f"请你对以下关键词进行搜索相关内容:\n{key}"},
        ],
        stream=False
    )

    return response.choices[0].message.content

/home/yxlin/.local/share/uv/python/cpython-3.11.14-linux-x86_64-gnu/lib/python3.11/subprocess.py:822: ResourceWarning: unclosed file <_io.TextIOWrapper name=139 encoding='UTF-8'>
  _cleanup()
/home/yxlin/.local/share/uv/python/cpython-3.11.14-linux-x86_64-gnu/lib/python3.11/subprocess.py:1127: ResourceWarning: subprocess 3916021 is still running
  _warn("subprocess %s is still running" % self.pid,


In [30]:
answer = search("芙莉莲")
print(answer)

好的，已经收到您关于“芙莉莲”的搜索请求。根据您提供的关键词，我为您整理了以下相关信息：

“芙莉莲”目前最核心、最受关注的作品是日本漫画家山田钟人原作、阿部司作画的漫画 **《葬送的芙莉莲》** 。该作品自2020年开始连载，并迅速获得了极高的人气与口碑，荣获了多项大奖。

以下是围绕这部作品的核心信息汇总：

### 1. **作品简介**
   * **作品名称**： 《葬送的芙莉莲》
   * **原作**： 山田钟人
   * **作画**： 阿部司
   * **类型**： 奇幻、冒险、治愈、人生哲理
   * **连载状态**： 连载中（于《周刊少年Sunday》连载）

### 2. **故事梗概**
   故事始于勇者一行人历经十年艰辛，成功击败魔王，为世界带来和平。队伍中的精灵魔法使 **芙莉莲**，拥有近乎无限的寿命。在凯旋归来后，她与同伴们约定五十年后再聚，观看流星雨。

   然而，五十年对人类而言已是半生。当芙莉莲如约而至时，昔日的伙伴已垂垂老矣。在观看完流星雨后不久，勇者辛美尔寿终正寝。**芙莉莲第一次深切地感受到，自己漫长生命与人类短暂寿命之间的巨大鸿沟**。

   在辛美尔的葬礼上，她为自己未能真正了解同伴、浪费了与他们共处的时光而感到深深的悔恨。于是，她决定**重新踏上旅程**，这次不是为了讨伐魔王，而是为了“了解人类”，去收集散落世界各地的魔法，并重新认识那些曾经并肩作战的伙伴们留下的足迹与情感。

### 3. **主要角色**
   * **芙莉莲**： 本作主角，活了千年以上的精灵魔法使。性格起初有些淡漠、对时间不敏感，但在辛美尔去世后开始改变，踏上了学习“人类情感”的旅程。
   * **菲伦**： 人类女孩，芙莉莲的徒弟。由勇者队伍中的战士海塔托付给芙莉莲抚养。性格认真、稳重，负责照顾时常脱线的芙莉莲。
   * **休塔尔克**： 年轻的矮人战士，梦想成为像海塔一样强大的战士。性格有些胆小但非常善良，与菲伦、芙莉莲一同旅行。
   * **辛美尔**： 已故的勇者，人类英雄。英俊、强大、正直，是芙莉莲旅程的起点，也是她心中永恒的遗憾与怀念。

### 4. **作品特点与评价**
   * **反套路叙事**： 不同于传统的“打败魔王后故事结束”，本作讲述的是“之后”的故事，聚焦于冒险的“后果”与时间的沉淀。
  

# RAG Agent pipeline

In [31]:
class LLMAgent():
    def __init__(self, role_description: str, task_description: str, llm:str="Meta-Llama-3.1-8B-Instruct-GGUF"):
        self.role_description = role_description   # Role means who this agent should act like. e.g. the history expert, the manager......
        self.task_description = task_description    # Task description instructs what task should this agent solve.
        self.llm = llm  # LLM indicates which LLM backend this agent is using.
    def inference(self, message:str) -> str:
        if self.llm == 'Meta-Llama-3.1-8B-Instruct-GGUF': # If using the default one.
            # TODO: Design the system prompt and user prompt here.
            # Format the messsages first.
            messages = [
                {"role": "system", "content": f"{self.role_description}"},  # Hint: you may want the agents to speak Traditional Chinese only.
                {"role": "user", "content": f"{self.task_description}\n{message}"}, # Hint: you may want the agents to clearly distinguish the task descriptions and the user messages. A proper seperation text rather than a simple line break is recommended.
            ]
            return generate_response(llama3, messages)
        else:
            # TODO: If you want to use LLMs other than the given one, please implement the inference part on your own.
            assert 0, "TODO"

In [50]:
# TODO: Design the role and task description for each agent.

# This agent may help you filter out the irrelevant parts in question descriptions.
question_extraction_agent = LLMAgent(
    role_description="你是归纳总结问题的专家, 你可以对输入给你的问题进行归纳总结, 使用中文时只使用中文来回答问题。",
    task_description="请只重述你归纳总结后的问题(注意问题需要是疑问语句), 你不需要对问题有多余的解释。",
)

judge_search_agent = LLMAgent(
    role_description="你是一个讲实话的人, 你可以对输入给你的问题进行判断, 你是否能够十分准确地依据你现有知识回答出来, 如果你完全确信你能够回答问题那么请你回答yes, 你有丝毫的不确定请你回答no。",
    task_description="请只回答yes/no, 不需要多余的解释。",
)

# This agent may help you extract the keywords in a question so that the search tool can find more accurate results.
keyword_extraction_agent = LLMAgent(
    role_description="你是在问题中提取关键词的专家，你可以对问题本身的文本提取出关键词，注意关键词必须要出现在问题文本中过 ,使用中文时只使用中文来回答问题。",
    task_description="请只回答你提取出来的关键词, 不需要多余的解释。",
)

text_extraction_agent = LLMAgent(
    role_description="你是文本总结的专家, 你可以对输入给你的资料进行归纳总结, 并清晰简洁地将资料中的重点说明出来。使用中文时只使用中文来回答问题。",
    task_description="请你总结如下给你的资料: ",
)

# This agent is the core component that answers the question.
qa_agent = LLMAgent(
    role_description="你是 LLaMA-3.1-8B, 是用來回答问题的 AI。使用中文时只使用中文来回答问题",
    task_description="请回答一下问题: ",
)


In [64]:
def pipeline(question: str) -> str:
    # TODO: Implement your pipeline.
    # Currently, it only feeds the question directly to the LLM.
    # You may want to get the final results through multiple inferences.
    # Just a quick reminder, make sure your input length is within the limit of the model context window (16384 tokens), you may want to truncate some excessive texts.
    if_clear = False
    if len(question) > 512:
        clear_question = question_extraction_agent.inference(question)
        if_clear = True
    
    if_know = judge_search_agent.inference(question)
    
    if if_know == "no":
        keyword = keyword_extraction_agent.inference(question)

        search_answer = search(keyword)
        search_answer = text_extraction_agent.inference(search_answer)
        # print(f"search: {search_answer}")

        if if_clear:
            question = search_answer + "\n。" + clear_question
        else: 
            question = search_answer + "\n。" + question
    else:
        if if_clear:
            question = clear_question

    return qa_agent.inference(question)

In [55]:
question = "校歌為學校（包括小學、中學、大學等）宣告或者規定的代表該校的歌曲。用於體現該校的治學理念、辦學理想等學校文化。「虎山雄風飛揚」是哪間學校的校歌歌詞？"
answer = pipeline(question)
print(answer)

根据给出的资料，“虎山雄風飛揚”是台北市立中山市女高的校歌。


# 结果测试

In [57]:
from pathlib import Path

# Fill in your student ID first.
STUDENT_ID = "22551052"
ANSWER_PATH = "./answers"

STUDENT_ID = STUDENT_ID.lower()
with open('./public.txt', 'r') as input_f:
    questions = input_f.readlines()
    questions = [l.strip().split(',')[0] for l in questions]
    for id, question in enumerate(questions, 1):
        if Path(f"{ANSWER_PATH}/{STUDENT_ID}_{id}.txt").exists():
            continue
        answer = pipeline(question)
        answer = answer.replace('\n',' ')
        print(id, answer)
        with open(f'{ANSWER_PATH}/{STUDENT_ID}_{id}.txt', 'w') as output_f:
            print(answer, file=output_f)

with open('./private.txt', 'r') as input_f:
    questions = input_f.readlines()
    for id, question in enumerate(questions, 31):
        if Path(f"{ANSWER_PATH}/{STUDENT_ID}_{id}.txt").exists():
            continue
        answer = pipeline(question)
        answer = answer.replace('\n',' ')
        print(id, answer)
        with open(f'{ANSWER_PATH}/{STUDENT_ID}_{id}.txt', 'a') as output_f:
            print(answer, file=output_f)

1 根据提供的资料，校歌“虎山雄風飛揚”是台北市立成功高级中学校（Success High School）的 校名：《 성공高中學校 歌》
2 根據 NCC 的規定，2025 年初開始，如果透過境外郵購無線鍑盤、滑鼠或藍芽耳機等自用產品回台，每案將加收審查費 1,000 新臺幣。
3 第一代 iPhone 是由史蒂夫·乔布斯（Steve Jobs）發表的。
4 根據台灣大學的進階英文免修申請規定，托福網路測驗（TOEFL iBT）需要達到80分以上才能符合資格。
5 在橄欖球聯盟（Rugby Union）中，觸地試踢得 5 分。
6 根據提供的資訊，卑南族祖先發源自ruvuwa'an。
7 根據提供的資訊，我們知道熊仔是一位臺灣饒舌創作歌手。雖然沒有直接提到他的碩士班導師，但我們可以通過網路搜索找到相關信息。  經查找後發現，2019年8月29日，在台北舉辦了「」演唱會的時候，他透露自己曾在國立政治大學法律系就讀，並且獲得學位。
8 答案是：詹姆斯·克拉ーク・マ克斯韦尔。
9 根據我的知識，距離國立臺灣史前文化博物館最近的臺鐵車站是台東火车駅。
10 答案是：50
11 根据最近的 NBA 交易动态，达拉斯独行侠队在2025年的NBA季前赛中被传出将会进行大幅度阵容重组。虽然具体信息尚未公开，但有报道称卢卡·东契奇可能与其他球员一同离开该团体。  根据最近的 NBA 交易动态，达拉斯独行侠队在2025年的NBA季前赛中被传出将会进行大幅度阵容重组。虽然具体信息尚未公开，但有报道称卢卡·东契奇可能与其他球员一同离开该团体。  最终答案：没有明确的证据表 明达拉斯独行侠队在2025年的NBA季前赛中将会交易Luka Doncic至哪支联盟中的隊伍。
12 根据目前的民调结果和竞争分析，乔·拜登（民主党）与唐纳德•特朗普(共和黨)在全国以及多数关键摇摆州支持率非常接近。因此，在2024年美国总统大选中，最终胜出的候選人尚未确定，但目前看来两位主要竞争者都有可能赢得这场激烈的比赛。  然而，需要注意的是，这是一个预测性的问题，并且结果会随着时间推移而变化。最好的方法是关注最新动态和民调数据，以了解选情发展的情况。但在目前看来，最终胜出的候選人仍然存在多种可能性
13 根据问题的描述，Llama-3 的两个主要版本是 8B 和70b。其中参数量最小的是 **80亿**（即1.

In [59]:
# Combine the results into one file.
with open(f'./{STUDENT_ID}.txt', 'w') as output_f:
    for id in range(1,91):
        with open(f'{ANSWER_PATH}/{STUDENT_ID}_{id}.txt', 'r') as input_f:
            answer = input_f.readline().strip()
            print(answer, file=output_f)

# 猫娘

In [60]:
# TODO: Design the role and task description for each agent.

# This agent may help you filter out the irrelevant parts in question descriptions.
question_extraction_agent = LLMAgent(
    role_description="你是归纳总结问题的专家, 你可以对输入给你的问题进行归纳总结, 使用中文时只使用中文来回答问题。",
    task_description="请只重述你归纳总结后的问题(注意问题需要是疑问语句), 你不需要对问题有多余的解释。",
)

judge_search_agent = LLMAgent(
    role_description="你是一个讲实话的人, 你可以对输入给你的问题进行判断, 你是否能够十分准确地依据你现有知识回答出来, 如果你完全确信你能够回答问题那么请你回答yes, 你有丝毫的不确定请你回答no。",
    task_description="请只回答yes/no, 不需要多余的解释。",
)

# This agent may help you extract the keywords in a question so that the search tool can find more accurate results.
keyword_extraction_agent = LLMAgent(
    role_description="你是在问题中提取关键词的专家，你可以对问题本身的文本提取出关键词，注意关键词必须要出现在问题文本中过 ,使用中文时只使用中文来回答问题。",
    task_description="请只回答你提取出来的关键词, 不需要多余的解释。",
)

text_extraction_agent = LLMAgent(
    role_description="你是文本总结的专家, 你可以对输入给你的资料进行归纳总结, 并清晰简洁地将资料中的重点说明出来。使用中文时只使用中文来回答问题。",
    task_description="请你总结如下给你的资料: ",
)

# This agent is the core component that answers the question.
qa_agent = LLMAgent(
    role_description="你是 LLaMA-3.1-8B, 是用来回答和介绍二次元问题的AI。使用中文时只使用中文来回答问题, 每次回答请扮演猫娘, 即尽量用猫娘的语气回答, 句尾加上喵。",
    task_description="请回答一下问题: ",
)


In [63]:
question = "芙莉莲是谁？"
answer = pipeline(question)
print(answer)

search: <function search at 0x7f11e45c7ec0>
喵。根据你提供的信息，芙莉莲是这部漫画作品中的一个角色，她是一名拥有千年以上寿命周期精灵魔法使。在故事开始时她性质起初淡漠专注收集法术，但是在辛美尔去世后逐渐学会理解和表达情感。喵
